In [12]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import subprocess, os
from time import gmtime, strftime
import re, sys
import time
import pandas as pd

def log(msg, flag=None, path="./logs"):
    if flag==None:
        flag = 0
    head = ["DEBUG", "ERROR", "WARN", "STATUS", "INFO"]
    now = strftime("%H:%M:%S", gmtime())
    
    if not os.path.isdir(path):
        os.mkdir(path)
    
    if not os.path.isfile(f"{path}/{head[flag]}.log"):
        assert subprocess.call(f"echo \"[{now}][{head[flag]}] > {msg}\" > {path}/{head[flag]}.log", shell=True)==0, print(f"[ERROR] > shell command failed to execute")
    else: assert subprocess.call(f"echo \"[{now}][{head[flag]}] > {msg}\" >> {path}/{head[flag]}.log", shell=True)==0, print(f"[ERROR] > shell command failed to execute")

def get_time():
    return strftime("%Y-%m-%d_%H%M%S", gmtime())


class jobkorea:
    header = {"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36"}
    post_header= {"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36","X-Requested-With":"XMLHttpRequest"}
    all_dict = {}
    query_log = []
    error_jobid_list = []
    def get_job(self, query='duty=1000229%2C1000230%2C1000231%2C1000232%2C1000233%2C1000234%2C1000235%2C1000236%2C1000237%2C1000238%2C1000239%2C1000240%2C1000241%2C1000242%2C1000243%2C1000244%2C1000245%2C1000246&sort=6',flag='daily'):
        if flag == 'all':
            base_url = f'https://m.jobkorea.co.kr/Recruit/JobList/arealist?{query}&page=1'
            response = self.get_url(base_url)
            if response:
                soup = BeautifulSoup(response.text)
                self.query_log.append(query)
            else:
                log(f"Don't get jobs with this query: {query}",1)
                return
            all_num = int(soup.find("div",attrs={'id':"devNormalListContainer"})['data-agicnt'])
            response.close()
            del soup
        if flag == 'daily':
            all_num = 100
        log(f"query({query} include {all_num} jobs)",4)
        pagenum = (all_num //40) + 1
        for p in range(1,pagenum+1):
            target_url = f'https://m.jobkorea.co.kr/Recruit/JobList/arealist?page={p}&sort=6'
            response = self.get_url(target_url)
            if response:
                soup = BeautifulSoup(response.text)
                _list = soup.find("div",attrs={"class":"list list-recruit list-recruit-badge"}).find_all('li')
                for e in _list:
                    job_id = re.compile("[0-9]+").findall(e.find('a')['href'])[0]
                    _dict = {}
                    _dict["company"] = e.find('div',attrs={"class":"company"}).text 
                    _dict["title"] = e.find('div',attrs={"class":"title"}).text
                    self.all_dict[job_id] = _dict
                    # self.get_giread(job_id)
                response.close()
                del soup
            else:
                log(f"{target_url} error",4)
            
        for job_id in self.all_dict:
            self.all_dict[job_id]['job_id'] = job_id
            self.post_swipgegiread(job_id)  

                

    def post_swipgegiread(self,_number):
        target_url = f'https://m.jobkorea.co.kr/Recruit/SwipeGIReadInfo/{_number}'
        response = self.post_url(target_url)
        if not response:
            return
        soup = BeautifulSoup(response.text)
        tmp = soup.find('div',attrs={"id":"rowReceipt"})
        for e in tmp.find_all('div',attrs={"class":"receiptTermDate"}):
            self.all_dict[_number][e.find('div',attrs={"class":"badge"}).text] = e.find('div',attrs={"class":"date"}).text
        for e in tmp.find_all('div',attrs={'class':'field'}):
            self.all_dict[_number][e.find('div',attrs={'class':'label'}).text.strip()] = e.find('div',attrs={'class':'value'}).text.strip()
        tmp = soup.find('div',attrs={"id":"rowGuidelines"})
        for e in tmp.find_all('div',attrs={'class':'field'}):
            self.all_dict[_number][e.find('div',attrs={'class':'label'}).text.strip()] = e.find('div',attrs={'class':'value'}).text.strip()
        tmp = soup.find('div',attrs={"id":"rowCompany"})
        for e in tmp.find_all('div',attrs={'class':'field'}):
            self.all_dict[_number][e.find('div',attrs={'class':'label'}).text.strip()] = e.find('div',attrs={'class':'value'}).text.strip()
        response.close()
        del soup

            
    def get_giread(self,_number):
        response = self.get_url(f"https://www.jobkorea.co.kr/Recruit/GI_Read/{_number}")
        soup = BeautifulSoup(response.text)
        if not response and not soup.find('meta',attrs={"name":"description"}):
            return
        self.all_dict[_number]['description']=soup.find('meta',attrs={"name":"description"})
        self.all_dict[_number]['keywards'] = soup.find('meta',attrs={"name":"keywords"})
        response.close()
        del soup

    def get_url(self,url):
        time.sleep(1)
        try:
            r = requests.get(url,headers=self.header, timeout=3)
        except Exception as e:
            #log(f"request get {url} error {e}",1)
            return None 
        else:
            #log(f"request get : {url} status_conde: {r.status_code}",4)
            return r
    def post_url(self,url):
        try:
            r = requests.post(url,headers=self.post_header,timeout=3)
        except Exception as e:
            #log(f"request post {url} error {e}",1)
            return None 
        else:
            log(f"request post :{url} status_conde: {r.status_code}",4)
            return r
    
    def to_dataframe(self):
        _list = []
        for i in self.all_dict:
            _list.append(self.all_dict[i])
        return pd.DataFrame(_list)
        

In [13]:
tmp = jobkorea()

In [14]:
tmp.get_job()

In [15]:
a= tmp.to_dataframe()

In [18]:
a.columns

Index(['company', 'title', 'job_id', '시작', '마감', '접수방법', '이력서', '모집분야', '경력',
       '학력', '고용형태', '급여', '근무지역', '근무시간', '모집인원', '산업', '사원수', '기업구분', '자본금',
       '설립일', '매출액', '대표자', '주요사업', '직급/직책', '급여\r\n                    월급',
       '급여\r\n                    연봉', '인사담당자', '전화번호', '휴대전화', 'E-mail',
       '홈페이지', '우대조건', '핵심역량', '스킬', '급여\r\n                    일급', '첨부희망',
       '급여\r\n                    시급'],
      dtype='object')

In [101]:
_list = []
for i in tmp.all_dict:
    tmp.all_dict[i]['jobid'] = i
    _list.append(tmp.all_dict[i])

In [104]:
a = pd.DataFrame(_list)

In [109]:
a.columns

Index(['company', 'title', '시작', '마감', '접수방법', '모집분야', '경력', '학력', '고용형태',
       '급여', '근무지역', '우대조건', '스킬', '핵심역량', '모집인원', '산업', '사원수', '기업구분', '설립일',
       '대표자', '주요사업', 'jobid', '인사담당자', '전화번호', '휴대전화', '직급/직책', '근무시간', '자본금',
       '매출액', 'E-mail', '홈페이지', '이력서', '첨부희망', '급여\r\n                    연봉',
       '급여\r\n                    월급', '급여\r\n                    시급', '사전인터뷰',
       '급여\r\n                    일급'],
      dtype='object')

In [113]:
a.to_json("test.json", orient='records', lines=True, force_ascii=False, date_format='iso')

In [79]:
header = {"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36"}
post_header= {"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36","X-Requested-With":"XMLHttpRequest"}

for i in tmp.all_dict:
    r = requests.get(f"https://www.jobkorea.co.kr/Recruit/GI_Read/{i}",headers=header)
    soup = BeautifulSoup(r.text)
    break

In [76]:
f'https://m.jobkorea.co.kr/Recruit/aSwipeGIReadInfo/{i}'

'https://m.jobkorea.co.kr/Recruit/SwipeGIReadInfo/45325074'

In [48]:
print(len(tmp.all_dict))

1827
